In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import tqdm
import os
import wandb


# Hyperparameters
mb_size = 64
Z_dim = 1000
h_dim = 128
lr = 1e-3

# Load MNIST data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the 28x28 image to 784
])

train_dataset = datasets.MNIST(root='../MNIST', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=mb_size, shuffle=True)

X_dim = 784  # 28 x 28

# Xavier Initialization
def xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# Generator
class Generator(nn.Module):
    def __init__(self, z_dim, h_dim, x_dim):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(z_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, x_dim)
        self.apply(xavier_init)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        out = torch.sigmoid(self.fc2(h))
        return out

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, x_dim, h_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(x_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, 1)
        self.apply(xavier_init)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        out = torch.sigmoid(self.fc2(h))
        return out



# Training
def cGANTraining(G, D, loss_fn, train_loader):
    G.train()
    D.train()

    D_loss_real_total = 0
    D_loss_fake_total = 0
    G_loss_total = 0
    t = tqdm.tqdm(train_loader)

    for it, (X_real, labels) in enumerate(t):
        # Prepare real data
        X_real = X_real.float().to(device)

        # Sample noise and labels
        z = torch.randn(X_real.size(0), Z_dim).to(device)
        ones_label = torch.ones(X_real.size(0), 1).to(device)
        zeros_label = torch.zeros(X_real.size(0), 1).to(device)

        # ================= Train Discriminator =================
        G_sample = G(z)
        D_real = D(X_real)
        D_fake = D(G_sample.detach())

        D_loss_real = loss_fn(D_real, ones_label)
        D_loss_fake = loss_fn(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake
        D_loss_real_total += D_loss_real.item()
        D_loss_fake_total += D_loss_fake.item()

        D_solver.zero_grad()
        D_loss.backward()
        D_solver.step()

        # ================= Train Generator ====================
        z = torch.randn(X_real.size(0), Z_dim).to(device)
        G_sample = G(z)
        D_fake = D(G_sample)

        G_loss = loss_fn(D_fake, ones_label)
        G_loss_total += G_loss.item()

        G_solver.zero_grad()
        G_loss.backward()
        G_solver.step()

    # ================= Logging =================
    D_loss_real_avg = D_loss_real_total / len(train_loader)
    D_loss_fake_avg = D_loss_fake_total / len(train_loader)
    D_loss_avg = D_loss_real_avg + D_loss_fake_avg
    G_loss_avg = G_loss_total / len(train_loader)

    wandb.log({
        "D_loss_real": D_loss_real_avg,
        "D_loss_fake": D_loss_fake_avg,
        "D_loss": D_loss_avg,
        "G_loss": G_loss_avg
    })

    return G, D, G_loss_avg, D_loss_avg



def save_sample(G, epoch, mb_size, Z_dim):
    out_dir = "out_vanila_GAN2"
    G.eval()
    with torch.no_grad():
        z = torch.randn(mb_size, Z_dim).to(device)
        samples = G(z).detach().cpu().numpy()[:16]

    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    if not os.path.exists(f'{out_dir}'):
        os.makedirs(f'{out_dir}')

    plt.savefig(f'{out_dir}/{str(epoch).zfill(3)}.png', bbox_inches='tight')
    plt.close(fig)



########################### Main #######################################
wandb_log = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Instantiate models
G = Generator(Z_dim, h_dim, X_dim).to(device)
D = Discriminator(X_dim, h_dim).to(device)

# Optimizers
G_solver = optim.Adam(G.parameters(), lr=lr)
D_solver = optim.Adam(D.parameters(), lr=lr)

# Loss function
def my_bce_loss(preds, targets):
    return F.binary_cross_entropy(preds, targets)

#loss_fn = nn.BCEWithLogitsLoss()
loss_fn = my_bce_loss

if wandb_log:
    wandb.init(project="conditional-gan-mnist")

    # Log hyperparameters
    wandb.config.update({
        "batch_size": mb_size,
        "Z_dim": Z_dim,
        "X_dim": X_dim,
        "h_dim": h_dim,
        "lr": lr,
    })

best_g_loss = float('inf')  # Initialize best generator loss
save_dir = 'checkpoints'
os.makedirs(save_dir, exist_ok=True)

#Train epochs
epochs = 5
print("5 EPOCHS: ")
for epoch in range(epochs):
    G, D, G_loss_avg, D_loss_avg= cGANTraining(G, D, loss_fn, train_loader)

    print(f'epoch{epoch}; D_loss: {D_loss_avg:.4f}; G_loss: {G_loss_avg:.4f}')

    if G_loss_avg < best_g_loss:
        best_g_loss = G_loss_avg
        torch.save(G.state_dict(), os.path.join(save_dir, 'G_best.pth'))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D_best.pth'))
        print(f"Saved Best Models at epoch {epoch} | G_loss: {best_g_loss:.4f}")

    save_sample(G, epoch, mb_size, Z_dim)

print("DONE")
# Inference
# G.load_state_dict(torch.load('checkpoints/G_best.pth'))
# G.eval()

# save_sample(G, "best", mb_size, Z_dim)

100%|██████████| 9.91M/9.91M [00:00<00:00, 42.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.13MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 8.65MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.44MB/s]


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: emisde-2 (ertveh-4-lule-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

epoch2; D_loss: 0.0535; G_loss: 5.5215
Saved Best Models at epoch 2 | G_loss: 5.5215


100%|██████████| 938/938 [00:12<00:00, 72.95it/s]


epoch3; D_loss: 0.0894; G_loss: 5.6593


100%|██████████| 938/938 [00:12<00:00, 72.57it/s]


epoch4; D_loss: 0.1733; G_loss: 4.8484
Saved Best Models at epoch 4 | G_loss: 4.8484
DONE


In [ ]:
best_g_loss = float('inf')  # Initialize best generator loss
save_dir = 'checkpoints'
os.makedirs(save_dir, exist_ok=True)

#Train epochs
epochs = 10
print("10 EPOCHS: ")
for epoch in range(epochs):
    G, D, G_loss_avg, D_loss_avg= cGANTraining(G, D, loss_fn, train_loader)

    print(f'epoch{epoch}; D_loss: {D_loss_avg:.4f}; G_loss: {G_loss_avg:.4f}')

    if G_loss_avg < best_g_loss:
        best_g_loss = G_loss_avg
        torch.save(G.state_dict(), os.path.join(save_dir, 'G_best.pth'))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D_best.pth'))
        print(f"Saved Best Models at epoch {epoch} | G_loss: {best_g_loss:.4f}")

    save_sample(G, epoch, mb_size, Z_dim)

print("DONE")
best_g_loss = float('inf')  # Initialize best generator loss
save_dir = 'checkpoints'
os.makedirs(save_dir, exist_ok=True)

10 EPOCHS: 


100%|██████████| 938/938 [00:25<00:00, 36.13it/s]


epoch0; D_loss: 0.3020; G_loss: 4.3416
Saved Best Models at epoch 0 | G_loss: 4.3416


100%|██████████| 938/938 [00:18<00:00, 51.72it/s]


epoch1; D_loss: 0.3841; G_loss: 3.7982
Saved Best Models at epoch 1 | G_loss: 3.7982


...

epoch7; D_loss: 0.6965; G_loss: 2.6393
Saved Best Models at epoch 7 | G_loss: 2.6393


100%|██████████| 938/938 [00:18<00:00, 50.57it/s]


epoch8; D_loss: 0.6895; G_loss: 2.5483
Saved Best Models at epoch 8 | G_loss: 2.5483


100%|██████████| 938/938 [00:19<00:00, 48.51it/s]


epoch9; D_loss: 0.7135; G_loss: 2.4641
Saved Best Models at epoch 9 | G_loss: 2.4641
DONE


In [ ]:
best_g_loss = float('inf')  # Initialize best generator loss
save_dir = 'checkpoints'
os.makedirs(save_dir, exist_ok=True)

epochs = 50
print("50 EPOCHS: ")
for epoch in range(epochs):
    G, D, G_loss_avg, D_loss_avg= cGANTraining(G, D, loss_fn, train_loader)

    print(f'epoch{epoch}; D_loss: {D_loss_avg:.4f}; G_loss: {G_loss_avg:.4f}')

    if G_loss_avg < best_g_loss:
        best_g_loss = G_loss_avg
        torch.save(G.state_dict(), os.path.join(save_dir, 'G_best.pth'))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D_best.pth'))
        print(f"Saved Best Models at epoch {epoch} | G_loss: {best_g_loss:.4f}")

    save_sample(G, epoch, mb_size, Z_dim)

print("DONE")

50 EPOCHS: 


100%|██████████| 938/938 [00:21<00:00, 44.66it/s]


epoch0; D_loss: 0.7093; G_loss: 2.2676
Saved Best Models at epoch 0 | G_loss: 2.2676


100%|██████████| 938/938 [00:19<00:00, 48.77it/s]


epoch1; D_loss: 0.7099; G_loss: 2.2622
Saved Best Models at epoch 1 | G_loss: 2.2622


...

epoch47; D_loss: 0.4926; G_loss: 2.9395


100%|██████████| 938/938 [00:21<00:00, 42.88it/s]


epoch48; D_loss: 0.4902; G_loss: 2.9586


100%|██████████| 938/938 [00:20<00:00, 45.33it/s]


epoch49; D_loss: 0.4861; G_loss: 2.9701
DONE


In [5]:
#With BCE Logit loss as Loss function
import torch.nn.functional as F
def my_BCEL_loss(preds, targets):
    return F.binary_cross_entropy_with_logits(preds, targets)

#loss_fn = nn.BCEWithLogitsLoss()
loss_fn = my_BCEL_loss

best_g_loss = float('inf')  # Initialize best generator loss
save_dir = 'checkpoints'
os.makedirs(save_dir, exist_ok=True)

#Train epochs
epochs = 5
print("5 EPOCHS: ")
for epoch in range(epochs):
    G, D, G_loss_avg, D_loss_avg= cGANTraining(G, D, loss_fn, train_loader)

    print(f'epoch{epoch}; D_loss: {D_loss_avg:.4f}; G_loss: {G_loss_avg:.4f}')

    if G_loss_avg < best_g_loss:
        best_g_loss = G_loss_avg
        torch.save(G.state_dict(), os.path.join(save_dir, 'G_best.pth'))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D_best.pth'))
        print(f"Saved Best Models at epoch {epoch} | G_loss: {best_g_loss:.4f}")

    save_sample(G, epoch, mb_size, Z_dim)

print("DONE")

5 EPOCHS: 


100%|██████████| 938/938 [00:25<00:00, 36.57it/s]


epoch0; D_loss: 1.0119; G_loss: 0.6916
Saved Best Models at epoch 0 | G_loss: 0.6916


100%|██████████| 938/938 [00:19<00:00, 48.96it/s]


epoch1; D_loss: 1.0073; G_loss: 0.6929


...

epoch2; D_loss: 1.0068; G_loss: 0.6930


100%|██████████| 938/938 [00:20<00:00, 45.04it/s]


epoch3; D_loss: 1.0066; G_loss: 0.6931


100%|██████████| 938/938 [00:21<00:00, 43.61it/s]


epoch4; D_loss: 1.0065; G_loss: 0.6931
DONE


In [4]:
best_g_loss = float('inf')  # Initialize best generator loss
save_dir = 'checkpoints'
os.makedirs(save_dir, exist_ok=True)

#Train epochs
epochs = 10
print("10 EPOCHS: ")
for epoch in range(epochs):
    G, D, G_loss_avg, D_loss_avg= cGANTraining(G, D, loss_fn, train_loader)

    print(f'epoch{epoch}; D_loss: {D_loss_avg:.4f}; G_loss: {G_loss_avg:.4f}')

    if G_loss_avg < best_g_loss:
        best_g_loss = G_loss_avg
        torch.save(G.state_dict(), os.path.join(save_dir, 'G_best.pth'))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D_best.pth'))
        print(f"Saved Best Models at epoch {epoch} | G_loss: {best_g_loss:.4f}")

    save_sample(G, epoch, mb_size, Z_dim)

print("DONE")

10 EPOCHS: 


100%|██████████| 938/938 [00:12<00:00, 72.71it/s]


epoch0; D_loss: 0.2931; G_loss: 4.3655
Saved Best Models at epoch 0 | G_loss: 4.3655


100%|██████████| 938/938 [00:13<00:00, 72.07it/s]


epoch1; D_loss: 0.3556; G_loss: 3.8379
Saved Best Models at epoch 1 | G_loss: 3.8379


...

epoch7; D_loss: 0.6239; G_loss: 2.6591
Saved Best Models at epoch 7 | G_loss: 2.6591


100%|██████████| 938/938 [00:13<00:00, 71.66it/s]


epoch8; D_loss: 0.6614; G_loss: 2.5551
Saved Best Models at epoch 8 | G_loss: 2.5551


100%|██████████| 938/938 [00:12<00:00, 72.30it/s]


epoch9; D_loss: 0.6725; G_loss: 2.4337
Saved Best Models at epoch 9 | G_loss: 2.4337
DONE


In [6]:
best_g_loss = float('inf')  # Initialize best generator loss
save_dir = 'checkpoints'
os.makedirs(save_dir, exist_ok=True)

#Train epochs
epochs = 50
print("50 EPOCHS: ")
for epoch in range(epochs):
    G, D, G_loss_avg, D_loss_avg= cGANTraining(G, D, loss_fn, train_loader)

    print(f'epoch{epoch}; D_loss: {D_loss_avg:.4f}; G_loss: {G_loss_avg:.4f}')

    if G_loss_avg < best_g_loss:
        best_g_loss = G_loss_avg
        torch.save(G.state_dict(), os.path.join(save_dir, 'G_best.pth'))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D_best.pth'))
        print(f"Saved Best Models at epoch {epoch} | G_loss: {best_g_loss:.4f}")

    save_sample(G, epoch, mb_size, Z_dim)

print("DONE")

50 EPOCHS: 


100%|██████████| 938/938 [00:19<00:00, 47.58it/s]


epoch0; D_loss: 1.0065; G_loss: 0.6931
Saved Best Models at epoch 0 | G_loss: 0.6931


100%|██████████| 938/938 [00:20<00:00, 46.18it/s]


epoch1; D_loss: 1.0065; G_loss: 0.6931


...

epoch47; D_loss: 1.0675; G_loss: 0.6746


100%|██████████| 938/938 [00:23<00:00, 39.22it/s]


epoch48; D_loss: 1.0672; G_loss: 0.6751


100%|██████████| 938/938 [00:24<00:00, 38.84it/s]


epoch49; D_loss: 1.0698; G_loss: 0.6752
DONE
